In [ ]:
from mymath import *
from potentials import *
import torch
import plotly.graph_objects as go
import numpy as np
from ipywidgets import widgets

In [ ]:
domain_range = torch.tensor([[-1.10001,1.1],[-0.20001,0.2]])

# interactive potential field in the space

edges = torch.tensor([[-1., 0],[0, 0],[1, 0]])

sample_slider = widgets.IntSlider(
    value=200,
    min=100,
    max=1e4,
    step=1,
    description='Resolution:',
    continuous_update=False,
    readout_format='.0f'
)
eps_slider = widgets.FloatSlider(
    value=1e-1,
    min=1e-3,
    max=2,
    step=1e-3,
    description='dhat:',
    continuous_update=False,
    readout_format='.3e'
)
X_slider = widgets.FloatSlider(
    value=edges[1,0],
    min=-1,
    max=1,
    step=1e-3,
    description='X:',
    continuous_update=False,
    readout_format='.3e'
)
Y_slider = widgets.FloatSlider(
    value=edges[1,1],
    min=-0.1,
    max=0.1,
    step=1e-4,
    description='Y:',
    continuous_update=False,
    readout_format='.4e'
)

sliders = [eps_slider,X_slider,Y_slider,sample_slider]
container = widgets.VBox([
    widgets.HBox([eps_slider,X_slider,Y_slider,sample_slider])])

xs = torch.arange(domain_range[0,0], domain_range[0,1], step=1 / sample_slider.value)
ys = torch.arange(domain_range[1,0], domain_range[1,1], step=1 / sample_slider.value)

points = torch.zeros((ys.shape[0] * xs.shape[0], 2))
for i, x in enumerate(xs):
    for j, y in enumerate(ys):
        points[j * xs.shape[0] + i,:] = torch.tensor([xs[i], ys[j]])

potential = convergent_ipc_potential(points, edges, eps_slider.value).view(ys.shape[0], xs.shape[0])
potential = torch.log(torch.maximum(potential, torch.ones_like(potential) * 1e-5))

trace1 = go.Scatter(x=edges[:,0], y=edges[:,1])
major_max = percentile(potential, 0.95)
trace2 = go.Contour(z=potential,x=xs,y=ys, 
    contours=dict(
        start=0,
        end=major_max,
        size=major_max/20,
    ))
g = go.FigureWidget(data=[trace1, trace2],
                    layout=go.Layout(width=800, height=600))

def response(change):
    edges[1,0] = X_slider.value
    edges[1,1] = Y_slider.value
    potential = convergent_ipc_potential(points, edges, eps_slider.value).view(ys.shape[0], xs.shape[0])
    potential = torch.log(torch.maximum(potential, torch.ones_like(potential) * 1e-5))

    with g.batch_update():
        g.data[1].x = xs
        g.data[1].y = ys
        g.data[1].z = potential
        g.data[0].x = edges[:,0]
        g.data[0].y = edges[:,1]
        # g.data[1].contours = dict(
        #     start=0,
        #     end=major_max,
        #     size=major_max/20,
        # )

for s in sliders:
    s.observe(response, names="value")

widgets.VBox([container,
              g])